# Coding Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In order to import the utils to this, we convert utils and networks to python.

In [ ]:
!jupyter nbconvert --to python '/content/drive/MyDrive/ECE1512/ProjectB/networks.ipynb'
%run '/content/drive/MyDrive/ECE1512/ProjectB/networks.ipynb'
!jupyter nbconvert --to python '/content/drive/MyDrive/ECE1512/ProjectB/utils.ipynb'
%run '/content/drive/MyDrive/ECE1512/ProjectB/utils.ipynb'
!jupyter nbconvert --to python '/content/drive/MyDrive/ECE1512/ProjectB/reparam_module.ipynb'
%run '/content/drive/MyDrive/ECE1512/ProjectB/reparam_module.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/ECE1512/ProjectB/networks.ipynb to python
[NbConvertApp] Writing 19903 bytes to /content/drive/MyDrive/ECE1512/ProjectB/networks.py
[NbConvertApp] Converting notebook /content/drive/MyDrive/ECE1512/ProjectB/utils.ipynb to python
[NbConvertApp] Writing 27468 bytes to /content/drive/MyDrive/ECE1512/ProjectB/utils.py


<ipython-input-2-7189dddfc568>:9: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import rotate as scipyrotate


[NbConvertApp] Converting notebook /content/drive/MyDrive/ECE1512/ProjectB/reparam_module.ipynb to python
[NbConvertApp] Writing 6844 bytes to /content/drive/MyDrive/ECE1512/ProjectB/reparam_module.py
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import tqdm
import os
import copy
import time
from torchvision.utils import save_image
import warnings

import random

warnings.filterwarnings('ignore')

import sys
sys.path.append('/content/drive/MyDrive/ECE1512/ProjectB')
from reparam_module import ReparamModule
import utils
import networks


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install kornia
# import wandb
import torchvision
import kornia as K
import torch.nn.functional as F

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 9.0 MB/s eta 0:00:00


# Load MNIST Data


In [ ]:
mnist_dataset = 'MNIST'
mnist_data_path = './mnist_data'
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(mnist_dataset, mnist_data_path)


100%|██████████| 9912422/9912422 [00:00<00:00, 245361152.72it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 26699513.74it/s]


Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 76126799.97it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 3864204.62it/s]


Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



# GATM Implementation

#### Argument Class Created

In [ ]:
NET = 'ConvNet'
class datm_arguments():
  def __init__(self,device,ipc = 10):
    self.dataset = 'MNIST'
    self.subset = 'imagenette'
    self.model = NET
    self.lr_img = 1000
    self.lr_lr = 1e-05
    self.lr_teacher = 0.01
    self.lr_y = 2
    self.Momentum_y = 0.9
    self.lr_init = 0.01
    self.batch_real = 256
    self.batch_train = 256
    self.batch_syn = None
    self.pix_init = "samples_predicted_correctly"

    self.data_path = 'mnist_data'
    self.buffer_path = './MNISTbuffers'
    self.expert_epochs = 2
    self.syn_steps = 22
    self.max_start_epoch = 15
    self.load_all = False
    self.no_aug = False
    self.zca = False
    self.texture = False
    self.canvas_size = 2
    self.canvas_samples = 1
    self.max_files = None
    self.max_experts = None
    self.force_save = False
    self.ema_decay = 0.999
    self.ipc = ipc
    self.eval_mode = 'S'
    self.num_eval = 20
    self.eval_it = 20
    self.epoch_eval_train = 200
    self.Iteration = 100
    self.Sequential_Generation = True
    self.expansion_end_epoch = 100
    self.current_max_start_epoch = 20
    self.min_start_epoch = 0
    self.max_start_epoch = 25
    self.threshold = 1.0
    self.Initialize_Label_With_Another_Model = False
    self.parall_eva = False
    self.Label_Model_Timestamp = -1



    self.num_classes = num_classes
    self.channel = channel
    self.im_size = im_size
    self.mean = mean
    self.std = std

    self.device = device
    self.dsa = True
    self.dsa_param = utils.ParamDiffAug()


#### Functions
**get_dataset**: there are more items will be returned, like classmap,
class_map_ive, loader_train_direct.

**Epoch**: the label would be long instead float and input new train_criterion

**evaluate_synset**: input the cirterion and pass to epoch.  

In [ ]:
"""
@misc{guo2023lossless,
  title={Towards Lossless Dataset Distillation via Difficulty-Aligned Trajectory Matching},
  author={Ziyao Guo and Kai Wang and George Cazenavette and Hui Li and Kaipeng Zhang and Yang You},
  year={2023},
  eprint={2310.05773},
  archivePrefix={arXiv},
  primaryClass={cs.CV}
}
"""
def get_dataset(dataset, data_path, batch_size=1, subset="imagenette", args=None, baseline=False):

  class_map = None
  loader_train_dict = None
  class_map_inv = None

  channel = 1
  im_size = (28, 28)
  num_classes = 10
  mean = [0.1307]
  std = [0.3081]
  if not args.zca:
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
  else:
    transform = transforms.Compose([transforms.ToTensor()])
  dst_train = datasets.MNIST(data_path, train=True, download=True, transform=transform) # no augmentation
  dst_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)
  class_names = [str(c) for c in range(num_classes)]
  class_map = {x:x for x in range(num_classes)}

  if args.zca:
    images = []
    labels = []
    print("Train ZCA")
    for i in tqdm.tqdm(range(len(dst_train))):
        im, lab = dst_train[i]
        images.append(im)
        labels.append(lab)
    images = torch.stack(images, dim=0).to(args.device)
    labels = torch.tensor(labels, dtype=torch.long, device="cpu")
    zca = K.enhance.ZCAWhitening(eps=0.1, compute_inv=True)
    zca.fit(images)
    zca_images = zca(images).to("cpu")
    dst_train = utils.TensorDataset(zca_images, labels)

    images = []
    labels = []
    print("Test ZCA")
    for i in tqdm.tqdm(range(len(dst_test))):
        im, lab = dst_test[i]
        images.append(im)
        labels.append(lab)
    images = torch.stack(images, dim=0).to(args.device)
    labels = torch.tensor(labels, dtype=torch.long, device="cpu")

    zca_images = zca(images).to("cpu")
    dst_test = utils.TensorDataset(zca_images, labels)

    print(type(zca))


    args.zca_trans = zca


  testloader = torch.utils.data.DataLoader(dst_test, batch_size=256, shuffle=False, num_workers=2)


  return channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader, loader_train_dict, class_map, class_map_inv


In [ ]:
"""
@misc{guo2023lossless,
  title={Towards Lossless Dataset Distillation via Difficulty-Aligned Trajectory Matching},
  author={Ziyao Guo and Kai Wang and George Cazenavette and Hui Li and Kaipeng Zhang and Yang You},
  year={2023},
  eprint={2310.05773},
  archivePrefix={arXiv},
  primaryClass={cs.CV}
}
"""
class Config:
    imagenette = [0, 217, 482, 491, 497, 566, 569, 571, 574, 701]

    # ["australian_terrier", "border_terrier", "samoyed", "beagle", "shih-tzu", "english_foxhound", "rhodesian_ridgeback", "dingo", "golden_retriever", "english_sheepdog"]
    imagewoof = [193, 182, 258, 162, 155, 167, 159, 273, 207, 229]

    # ["tabby_cat", "bengal_cat", "persian_cat", "siamese_cat", "egyptian_cat", "lion", "tiger", "jaguar", "snow_leopard", "lynx"]
    imagemeow = [281, 282, 283, 284, 285, 291, 292, 290, 289, 287]

    # ["peacock", "flamingo", "macaw", "pelican", "king_penguin", "bald_eagle", "toucan", "ostrich", "black_swan", "cockatoo"]
    imagesquawk = [84, 130, 88, 144, 145, 22, 96, 9, 100, 89]

    # ["pineapple", "banana", "strawberry", "orange", "lemon", "pomegranate", "fig", "bell_pepper", "cucumber", "green_apple"]
    imagefruit = [953, 954, 949, 950, 951, 957, 952, 945, 943, 948]

    # ["bee", "ladys slipper", "banana", "lemon", "corn", "school_bus", "honeycomb", "lion", "garden_spider", "goldfinch"]
    imageyellow = [309, 986, 954, 951, 987, 779, 599, 291, 72, 11]

    dict = {
        "imagenette" : imagenette,
        "imagewoof" : imagewoof,
        "imagefruit": imagefruit,
        "imageyellow": imageyellow,
        "imagemeow": imagemeow,
        "imagesquawk": imagesquawk,
    }

config = Config()

def epoch(mode, dataloader, net, optimizer, criterion, args, aug, texture=False, If_Float = False):
    loss_avg, acc_avg, num_exp = 0, 0, 0
    if args.parall_eva==False:
        device = torch.device("cuda:0")
    else:
        device = args.device

    if args.dataset == "ImageNet":
        class_map = {x: i for i, x in enumerate(config.img_net_classes)}

    if mode == 'train':
        net.train()
    else:
        net.eval()
    net = net.to(device)
    for i_batch, datum in enumerate(dataloader):
        img = datum[0].float().to(device)
        if If_Float:
            lab = datum[1].float().to(device)
        else:
            lab = datum[1].long().to(device)
        if aug:
            if args.dsa:
                img = utils.DiffAugment(img, args.dsa_strategy, param=args.dsa_param)
            else:
                img = utils.augment(img, args.dc_aug_param, device=device)
        img = img.to(device)
        if args.dataset == "ImageNet" and mode != "train":
            lab = torch.tensor([class_map[x.item()] for x in lab]).to(device)

        n_b = lab.shape[0]

        output = net(img)
        loss = criterion(output, lab)


        if If_Float:
            acc = 1.
        else:
            acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))

        loss_avg += loss.item()*n_b
        acc_avg += acc
        num_exp += n_b

        if mode == 'train':
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg


def evaluate_synset(it_eval, net, images_train, labels_train, testloader, args, return_loss=False, texture=False, train_criterion=None, Preciser_Scheduler=False, type=1):
    if args.parall_eva==False:
        device = torch.device("cuda:0")
    else:
        device = args.device
    net = net.to(device)
    images_train.to(device)
    labels_train.to(device)
    lr = float(args.lr_net)
    Epoch = int(args.epoch_eval_train)

    if Preciser_Scheduler:
        LR_begin=0.0000000001
        LR_End = float(args.lr_net)
        if type==0:
            t=0
        else:
            t=500
        T=Epoch
        lambda1 = lambda epoch: ((LR_End-LR_begin)*epoch / t) if epoch < t else  LR_End * (1+math.cos(math.pi*(epoch - t)/(T-t)))/2.
        optimizer = torch.optim.Adam(net.parameters(), lr=LR_End, weight_decay=0.0005)
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
    else:
        lr_schedule = [Epoch//2+1]
        optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)

    '''test'''
    test_criterion = nn.CrossEntropyLoss().to(device)
    If_Float = True
    if train_criterion == None:
        train_criterion = nn.CrossEntropyLoss().to(device)
        If_Float = False

    dst_train = utils.TensorDataset(images_train, labels_train)
    trainloader = torch.utils.data.DataLoader(dst_train, batch_size=args.batch_train, shuffle=True, num_workers=0)

    start = time.time()
    acc_train_list = []
    loss_train_list = []

    for ep in tqdm.tqdm(range(Epoch+1)):
        loss_train, acc_train = epoch('train', trainloader, net, optimizer, train_criterion, args, aug=True, texture=texture, If_Float=If_Float)
        acc_train_list.append(acc_train)
        loss_train_list.append(loss_train)
        if ep == Epoch:
            with torch.no_grad():
                loss_test, acc_test = epoch('test', testloader, net, optimizer, test_criterion, args, aug=False, If_Float = False)
        if Preciser_Scheduler:
            scheduler.step()
        else:
            if ep in lr_schedule:
                lr *= 0.1
                optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)


    time_train = time.time() - start

    print('%s Evaluate_%02d: epoch = %04d train time = %d s train loss = %.6f train acc = %.4f, test acc = %.4f' % (utils.get_time(), it_eval, Epoch, int(time_train), loss_train, acc_train, acc_test))

    if return_loss:
        return net, acc_train_list, acc_test, loss_train_list, loss_test
    else:
        return net, acc_train_list, acc_test

#### DATM Function
**Reference**: G. Cazenavette, T. Wang, A. Torralba, A. A. Efros, and J.-Y. Zhu, "Dataset Distillation by Matching Training Trajectories," in Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition, pp. 4750-4759, 2022. [Online]. Available: https://arxiv.org/abs/2203.11932

In [ ]:
"""
@misc{guo2023lossless,
  title={Towards Lossless Dataset Distillation via Difficulty-Aligned Trajectory Matching},
  author={Ziyao Guo and Kai Wang and George Cazenavette and Hui Li and Kaipeng Zhang and Yang You},
  year={2023},
  eprint={2310.05773},
  archivePrefix={arXiv},
  primaryClass={cs.CV}
}
"""
def manual_seed(seed=0):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed)

def DATM (args):
  best_data = []
  r_test_acc = []
  r_train_acc = []
  last_data = []

  manual_seed()

  if args.texture and args.pix_init == "real":
      print("WARNING: Using texture with real initialization will take a very long time to smooth out the boundaries between images.")

  if args.max_experts is not None and args.max_files is not None:
      args.total_experts = args.max_experts * args.max_files

  if args.zca and args.texture:
    raise AssertionError("Cannot use zca and texture together")

  ''' basic setup '''
  args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
  args.dsa = True if args.dsa == 'True' else False
  args.dc_aug_param = utils.get_daparam(args.dataset, model, NET, ipc = args.ipc)
  args.dsa_strategy = args.dc_aug_param['strategy']

  eval_it_pool = np.arange(0, args.Iteration + 1, args.eval_it).tolist()
  #eval_it_pool = [0, 50, 100, 150, 170, 199]

  channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader, loader_train_dict, class_map, class_map_inv = get_dataset(args.dataset, args.data_path, args.batch_real, args.subset, args=args)
  model_eval_pool = utils.get_eval_pool(args.eval_mode, args.model, args.model)

  accs_all_exps = dict() # record performances of all experiments
  for key in model_eval_pool:
      accs_all_exps[key] = []

  data_save = []


  if args.dsa:
    args.dc_aug_param = None

  if args.zca:
    zca_trans = args.zca_trans
  else:
    zca_trans = None

  # wandb.init(sync_tensorboard=False,
  #       project="DatasetDistillation",
  #       job_type="CleanRepo",
  #       config=args,
  # )

  args.zca_trans = zca_trans

  if args.batch_syn is None:
    args.batch_syn = num_classes * args.ipc


  args.distributed = torch.cuda.device_count() > 1

  '''organization Dataset'''
  indices_class = [[] for c in range(num_classes)]
  images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
  labels_all = [dst_train[i][1] for i in range(len(dst_train))]

  for i, lab in enumerate(labels_all):
      indices_class[lab].append(i)

  images_all = torch.cat(images_all, dim=0).to(args.device)
  labels_all = torch.tensor(labels_all, dtype=torch.long, device=args.device)

  for c in range(num_classes):
      print('class c = %d: %d real images'%(c, len(indices_class[c])))

  def get_images(c, n): # get random n images from class c
      idx_shuffle = np.random.permutation(indices_class[c])[:n]
      return images_all[idx_shuffle]

  '''initialize the synthetic data '''
  label_syn = torch.tensor([np.ones(args.ipc)*i for i in range(num_classes)], dtype=torch.long, requires_grad=False, device=args.device).view(-1)
  image_syn = torch.randn(size=(num_classes * args.ipc, channel, im_size[0], im_size[1]), dtype=torch.float)
  syn_lr = torch.tensor(args.lr_teacher).to(args.device)
  expert_dir = os.path.join(args.buffer_path, args.dataset)
  expert_dir = os.path.join(expert_dir, args.model)
  print("Expert Dir: {}".format(expert_dir))

  if args.load_all:
    buffer = []
    n = 0
    while os.path.exists(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n))):
        buffer = buffer + torch.load(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n)))
        n += 1
    if n == 0:
        raise AssertionError("No buffers detected at {}".format(expert_dir))
  else:
    expert_files = []
    n = 0
    while os.path.exists(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n))):
        expert_files.append(os.path.join(expert_dir, "replay_buffer_{}.pt".format(n)))
        n += 1
    if n == 0:
        raise AssertionError("No buffers detected at {}".format(expert_dir))
    file_idx = 0
    expert_idx = 0
    # random.shuffle(expert_files)
    if args.max_files is not None:
        expert_files = expert_files[:args.max_files]

    expert_id = [i for i in range(len(expert_files))]
    random.shuffle(expert_id)

    print("loading file {}".format(expert_files[expert_id[file_idx]]))
    buffer = torch.load(expert_files[expert_id[file_idx]])
    if args.max_experts is not None:
        buffer = buffer[:args.max_experts]
    buffer_id = [i for i in range(len(buffer))]
    random.shuffle(buffer_id)


  if args.pix_init == 'real':
    print('initialize synthetic data from random real images')
    for c in range(args.num_classes):
      image_syn.data[c * args.ipc:(c + 1) * args.ipc] = get_images(c, args.ipc).detach().data
  elif args.pix_init == 'samples_predicted_correctly':
    if args.parall_eva==False:
        device = torch.device("cuda:0")
    else:
        device = args.device
    if args.Initialize_Label_With_Another_Model:
        Temp_net = utils.get_network(args.Initialize_Label_Model, channel, num_classes, im_size).to(device)  # get a random model
    else:
        Temp_net = utils.get_network(args.model, channel, num_classes, im_size).to(device)  # get a random model
    Temp_net.eval()
    Temp_net = ReparamModule(Temp_net)
    if args.distributed and args.parall_eva==True:
        Temp_net = torch.nn.DataParallel(Temp_net)
    Temp_net.eval()
    logits=[]
    label_expert_files = expert_files
    temp_params = torch.load(label_expert_files[0])[0][args.Label_Model_Timestamp]
    temp_params = torch.cat([p.data.to(device).reshape(-1) for p in temp_params], 0)

    if args.distributed and args.parall_eva==True:
        temp_params = temp_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
    for c in range(num_classes):
        data_for_class_c = get_images(c, len(indices_class[c])).detach().data
        n, _, w, h = data_for_class_c.shape
        selected_num = 0
        select_times = 0
        cur=0
        temp_img = None
        Wrong_Predicted_Img = None
        batch_size = 256
        index = []
        while len(index)<args.ipc:
            print(str(c)+'.'+str(select_times)+'.'+str(cur))
            current_data_batch = data_for_class_c[batch_size*select_times : batch_size*(select_times+1)].detach().to(device)
            if batch_size*select_times > len(data_for_class_c):
                select_times = 0
                cur+=1
                temp_params = torch.load(label_expert_files[int(cur/10)%10])[cur%10][args.Label_Model_Timestamp]
                temp_params = torch.cat([p.data.to(device).reshape(-1) for p in temp_params], 0).to(device)
                if args.distributed and args.parall_eva==True:
                    temp_params = temp_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
                continue
            logits = Temp_net(current_data_batch, flat_param=temp_params).detach()
            prediction_class = np.argmax(logits.cpu().data.numpy(), axis=-1)
            for i in range(len(prediction_class)):
                if prediction_class[i]==c and len(index)<args.ipc:
                    index.append(batch_size*select_times+i)
                    index=list(set(index))
            select_times+=1
            if len(index) == args.ipc:
                temp_img = torch.index_select(data_for_class_c, dim=0, index=torch.tensor(index).to(device))
                break
        image_syn.data[c * args.ipc:(c + 1) * args.ipc] = temp_img.detach()
  else:
    print('initialize synthetic data from random noise')

  ''' training part '''
  image_syn = image_syn.detach().to(args.device).requires_grad_(True)
  syn_lr = syn_lr.detach().to(args.device).requires_grad_(True)

  optimizer_img = torch.optim.SGD([image_syn], lr=args.lr_img, momentum=0.5)
  optimizer_lr = torch.optim.SGD([syn_lr], lr=args.lr_lr, momentum=0.5)
  optimizer_img.zero_grad()


  best_acc = {m: 0 for m in model_eval_pool}
  best_std = {m: 0 for m in model_eval_pool}

  def SoftCrossEntropy(inputs, target, reduction='average'):
      input_log_likelihood = -F.log_softmax(inputs, dim=1)
      target_log_likelihood = F.softmax(target, dim=1)
      batch = inputs.shape[0]
      loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
      return loss

  criterion = SoftCrossEntropy

  '''train'''
  if args.pix_init == "samples_predicted_correctly":
      if args.Initialize_Label_With_Another_Model:
          Temp_net = utils.get_network(args.Initialize_Label_Model, channel, num_classes, im_size).to(device)  # get a random model
      else:
          Temp_net = utils.get_network(args.model, channel, num_classes, im_size).to(device)  # get a random model
      Temp_net.eval()
      Temp_net = ReparamModule(Temp_net)
      if args.distributed:
          Temp_net = torch.nn.DataParallel(Temp_net)
      Temp_net.eval()
      logits=[]
      batch_size = 256
      for i in range(len(label_expert_files)):
          Temp_Buffer = torch.load(label_expert_files[i])
          for j in Temp_Buffer:
              temp_logits = None
              for select_times in range((len(image_syn)+batch_size-1)//batch_size):
                  current_data_batch = image_syn[batch_size*select_times : batch_size*(select_times+1)].detach().to(device)
                  Temp_params = j[args.Label_Model_Timestamp]
                  Initialize_Labels_params = torch.cat([p.data.to(args.device).reshape(-1) for p in Temp_params], 0)
                  if args.distributed:
                      Initialize_Labels_params = Initialize_Labels_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
                  Initialized_Labels = Temp_net(current_data_batch, flat_param=Initialize_Labels_params)
                  if temp_logits == None:
                      temp_logits = Initialized_Labels.detach()
                  else:
                      temp_logits = torch.cat((temp_logits, Initialized_Labels.detach()),0)
              logits.append(temp_logits.detach().cpu())
      logits_tensor = torch.stack(logits)
      true_labels = label_syn.cpu()
      predicted_labels = torch.argmax(logits_tensor, dim=2).cpu()
      correct_predictions = predicted_labels == true_labels.view(1, -1)
      mask = correct_predictions.unsqueeze(2)
      correct_logits = logits_tensor * mask.float()
      correct_logits_per_model = correct_logits.sum(dim=0)
      num_correct_images_per_model = correct_predictions.sum(dim=0, dtype=torch.float)
      average_logits_per_image = correct_logits_per_model / num_correct_images_per_model.unsqueeze(1)
      Initialized_Labels = average_logits_per_image

  elif args.pix_init == "real":
      Temp_net = utils.get_network(args.model, channel, num_classes, im_size).to(args.device)  # get a random model
      Temp_net = ReparamModule(Temp_net)
      if args.distributed:
          Temp_net = torch.nn.DataParallel(Temp_net)
      Temp_net.eval()
      Temp_params = buffer[0][-1]
      Initialize_Labels_params = torch.cat([p.data.to(args.device).reshape(-1) for p in Temp_params], 0)
      if args.distributed:
          Initialize_Labels_params = Initialize_Labels_params.unsqueeze(0).expand(torch.cuda.device_count(), -1)
      Initialized_Labels = Temp_net(image_syn, flat_param=Initialize_Labels_params)

  acc = np.sum(np.equal(np.argmax(Initialized_Labels.cpu().data.numpy(), axis=-1), label_syn.cpu().data.numpy()))
  print('InitialAcc:{}'.format(acc/len(label_syn)))

  label_syn = copy.deepcopy(Initialized_Labels.detach()).to(args.device).requires_grad_(True)
  label_syn.requires_grad=True
  label_syn = label_syn.to(args.device)


  optimizer_y = torch.optim.SGD([label_syn], lr=args.lr_y, momentum=args.Momentum_y)
  vs = torch.zeros_like(label_syn)
  accumulated_grad = torch.zeros_like(label_syn)
  last_random = 0

  del Temp_net

  for it in range(0, args.Iteration+1):
    save_this_it = False
    ''' Evaluate synthesis data '''
    if it == (args.Iteration-1):
        last_data.append([copy.deepcopy(image_save.detach().cpu()), copy.deepcopy(label_syn.detach().cpu())])
    if it in eval_it_pool:
      for model_eval in model_eval_pool:
        print('-------------------------\nEvaluation\nmodel_train = %s, model_eval = %s, iteration = %d'%(args.model, model_eval, it))
        if args.dsa:
            print('DSA augmentation strategy: \n', args.dsa_strategy)
            print('DSA augmentation parameters: \n', args.dsa_param.__dict__)
        else:
            print('DC augmentation parameters: \n', args.dc_aug_param)

        accs_test = []
        accs_train = []
        for it_eval in range(args.num_eval):
          net_eval = utils.get_network(model_eval, channel, num_classes, im_size).to(args.device) # get a random model

          eval_labs = label_syn
          with torch.no_grad():
              image_save = image_syn
          image_syn_eval, label_syn_eval = copy.deepcopy(image_save.detach()), copy.deepcopy(eval_labs.detach()) # avoid any unaware modification

          args.lr_net = syn_lr.item()
          _, acc_train, acc_test = evaluate_synset(it_eval, net_eval, image_syn_eval, label_syn_eval, testloader, args, train_criterion=criterion)
          accs_test.append(acc_test)
          accs_train.append(acc_train)

        accs_test = np.array(accs_test)
        accs_train = np.array(accs_train)
        acc_test_mean = np.mean(accs_test)
        acc_test_std = np.std(accs_test)
        r_test_acc.append((model_eval, np.mean(accs_test)))
        r_train_acc.append((model_eval, np.mean(accs_train)))



        if acc_test_mean > best_acc[model_eval]:
            best_acc[model_eval] = acc_test_mean
            best_std[model_eval] = acc_test_std
            save_this_it = True
        print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs_test), model_eval, acc_test_mean, acc_test_std))


      '''save dataset'''
      with torch.no_grad():
        image_save = image_syn.cuda()
        save_dir = os.path.join(".", "logged_files", args.dataset)

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        torch.save(image_save.cpu(), os.path.join(save_dir, "images_{}.pt".format(it)))
        torch.save(label_syn.cpu(), os.path.join(save_dir, "labels_{}.pt".format(it)))

        save_name = os.path.join(save_dir, args.pix_init+'vis_%s_%dipc_iter%d.png'%( args.dataset, args.ipc, it))
        image_syn_vis = copy.deepcopy(image_syn.detach().cpu())
        image_syn_vis[image_syn_vis<0] = 0.0
        image_syn_vis[image_syn_vis>1] = 1.0
        save_image(image_syn_vis, save_name, nrow=args.ipc)

        if save_this_it:
            save_name = os.path.join(save_dir, 'images_best.png')
            image_syn_vis = copy.deepcopy(image_syn.detach().cpu())
            image_syn_vis[image_syn_vis<0] = 0.0
            image_syn_vis[image_syn_vis>1] = 1.0
            save_image(image_syn_vis, save_name, nrow=args.ipc)
            best_data.append([copy.deepcopy(image_save.detach().cpu()), copy.deepcopy(label_syn.detach().cpu())])
            torch.save(image_save.cpu(), os.path.join(save_dir, "images_best.pt".format(it)))
            torch.save(label_syn.cpu(), os.path.join(save_dir, "labels_best.pt".format(it)))


        if args.ipc < 50 or args.force_save:
            upsampled = image_save
            if args.dataset != "ImageNet":
                upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
            grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)


            for clip_val in [2.5]:
                std = torch.std(image_save)
                mean = torch.mean(image_save)
                upsampled = torch.clip(image_save, min=mean-clip_val*std, max=mean+clip_val*std)
                if args.dataset != "ImageNet":
                  upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                  upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)


            if args.zca:
                image_save = image_save.to(args.device)
                image_save = args.zca_trans.inverse_transform(image_save)
                image_save.cpu()
                torch.save(image_save.cpu(), os.path.join(save_dir, "images_zca_{}.pt".format(it)))
                upsampled = image_save
                if args.dataset != "ImageNet":
                    upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                    upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)

                for clip_val in [2.5]:
                    std = torch.std(image_save)
                    mean = torch.mean(image_save)
                    upsampled = torch.clip(image_save, min=mean - clip_val * std, max=mean + clip_val * std)
                    if args.dataset != "ImageNet":
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=2)
                        upsampled = torch.repeat_interleave(upsampled, repeats=4, dim=3)
                    grid = torchvision.utils.make_grid(upsampled, nrow=10, normalize=True, scale_each=True)


    ''' training'''

    student_net = utils.get_network(args.model, channel, num_classes, im_size).to(args.device)  # get a random model
    student_net = ReparamModule(student_net)
    if args.distributed:
      student_net = torch.nn.DataParallel(student_net)
    student_net.train()
    num_params = sum([np.prod(p.size()) for p in (student_net.parameters())])

    if args.load_all:
      expert_trajectory = buffer[np.random.randint(0, len(buffer))]
    else:
      expert_trajectory = buffer[buffer_id[expert_idx]]
      expert_idx += 1
      if expert_idx == len(buffer):
          expert_idx = 0
          file_idx += 1
          if file_idx == len(expert_files):
              file_idx = 0
              random.shuffle(expert_files)
          print("loading file {}".format(expert_files[file_idx]))
          if args.max_files != 1:
              del buffer
              buffer = torch.load(expert_files[file_idx])
          if args.max_experts is not None:
              buffer = buffer[:args.max_experts]
          random.shuffle(buffer_id)

    if args.Sequential_Generation:
        Upper_Bound = args.current_max_start_epoch + int((args.max_start_epoch-args.current_max_start_epoch) * it/(args.expansion_end_epoch))
        Upper_Bound = min(Upper_Bound, args.max_start_epoch)
    else:
        Upper_Bound = args.max_start_epoch

    start_epoch = np.random.randint(args.min_start_epoch, Upper_Bound)

    starting_params = expert_trajectory[start_epoch]

    target_params = expert_trajectory[start_epoch+args.expert_epochs]
    target_params = torch.cat([p.data.to(args.device).reshape(-1) for p in target_params], 0)

    student_params = [torch.cat([p.data.to(args.device).reshape(-1) for p in starting_params], 0).requires_grad_(True)]

    starting_params = torch.cat([p.data.to(args.device).reshape(-1) for p in starting_params], 0)

    syn_images = image_syn

    y_hat = label_syn.to(args.device)
    param_loss_list = []
    param_dist_list = []
    indices_chunks = []

    for step in range(args.syn_steps):
      if not indices_chunks:
          indices = torch.randperm(len(syn_images))
          indices_chunks = list(torch.split(indices, args.batch_syn))

      these_indices = indices_chunks.pop()

      x = syn_images[these_indices]
      this_y = y_hat[these_indices]


      if args.dsa and (not args.no_aug):
          x = utils.DiffAugment(x, args.dsa_strategy, param=args.dsa_param)

      if args.distributed:
          forward_params = student_params[-1].unsqueeze(0).expand(torch.cuda.device_count(), -1)
      else:
          forward_params = student_params[-1]
      x = student_net(x, flat_param=forward_params)
      ce_loss = criterion(x, this_y)

      grad = torch.autograd.grad(ce_loss, student_params[-1], create_graph=True)[0]

      student_params.append(student_params[-1] - syn_lr * grad)

    param_loss = torch.tensor(0.0).to(args.device)
    param_dist = torch.tensor(0.0).to(args.device)

    param_loss += torch.nn.functional.mse_loss(student_params[-1], target_params, reduction="sum")
    param_dist += torch.nn.functional.mse_loss(starting_params, target_params, reduction="sum")

    param_loss_list.append(param_loss)
    param_dist_list.append(param_dist)

    param_loss /= num_params
    param_dist /= num_params

    param_loss /= param_dist

    grand_loss = param_loss

    optimizer_img.zero_grad()
    optimizer_lr.zero_grad()
    optimizer_y.zero_grad()

    grand_loss.backward()

    if grand_loss<=args.threshold:
      optimizer_y.step()
      optimizer_img.step()
      optimizer_lr.step()

    for _ in student_params:
        del _

    if it%10 == 0:
        print('%s iter = %04d, loss = %.4f' % (utils.get_time(), it, grand_loss.item()))
  return best_data, last_data, r_test_acc, r_train_acc




#### DATM in Two initiallzation Way

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
args = datm_arguments(device, 10)
model = utils.get_network(NET,channel,num_classes).to(device)
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(mnist_dataset, mnist_data_path)
NET = "ConvNet"
args.pix_init = "samples_predicted_correctly"
spc_best_data,spc_last_data, spc_r_test_acc,spc_r_train_acc = DATM(args)

class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images
Expert Dir: ./MNISTbuffers/MNIST/ConvNet
loading file ./MNISTbuffers/MNIST/ConvNet/replay_buffer_2.pt
0.0.0
1.0.0
2.0.0
3.0.0
4.0.0
5.0.0
6.0.0
7.0.0
8.0.0
9.0.0
InitialAcc:1.0
-------------------------
Evaluation
model_train = ConvNet, model_eval = ConvNet, iteration = 0
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.001, 'strategy': 'crop_scale_rotate'}


100%|██████████| 201/201 [00:12<00:00, 15.55it/s]


[2023-12-04 22:14:52] Evaluate_00: epoch = 0200 train time = 12 s train loss = 0.106107 train acc = 0.0100, test acc = 0.9551


100%|██████████| 201/201 [00:12<00:00, 16.71it/s]


[2023-12-04 22:15:04] Evaluate_01: epoch = 0200 train time = 12 s train loss = 0.114521 train acc = 0.0100, test acc = 0.9542


...

[2023-12-04 22:41:51] Evaluate_17: epoch = 0200 train time = 13 s train loss = 0.073377 train acc = 0.0100, test acc = 0.9609


100%|██████████| 201/201 [00:14<00:00, 13.84it/s]


[2023-12-04 22:42:05] Evaluate_18: epoch = 0200 train time = 14 s train loss = 0.068752 train acc = 0.0100, test acc = 0.9625


100%|██████████| 201/201 [00:12<00:00, 16.36it/s]


[2023-12-04 22:42:17] Evaluate_19: epoch = 0200 train time = 12 s train loss = 0.080454 train acc = 0.0100, test acc = 0.9642
Evaluate 20 random ConvNet, mean = 0.9603 std = 0.0025
-------------------------
[2023-12-04 22:42:19] iter = 0100, loss = 0.8997


In [ ]:
print(spc_r_test_acc)
print(spc_r_train_acc)

[('ConvNet', 0.9559100000000003), ('ConvNet', 0.9620600000000001), ('ConvNet', 0.964135), ('ConvNet', 0.96187), ('ConvNet', 0.9613100000000001), ('ConvNet', 0.9603099999999998)]
[('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01)]


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
args = datm_arguments(device, 10)
model = utils.get_network(NET,channel,num_classes).to(device)
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(mnist_dataset, mnist_data_path)
NET = "ConvNet"
args.pix_init = "real"
real_best_data, real_last_data, real_r_test_acc,real_r_train_acc = DATM(args)

class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images
Expert Dir: ./MNISTbuffers/MNIST/ConvNet
loading file ./MNISTbuffers/MNIST/ConvNet/replay_buffer_2.pt
initialize synthetic data from random real images
InitialAcc:0.99
-------------------------
Evaluation
model_train = ConvNet, model_eval = ConvNet, iteration = 0
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.001, 'strategy': 'crop_scale_rotate'}


100%|██████████| 201/201 [00:14<00:00, 13.45it/s]


[2023-12-04 23:16:00] Evaluate_00: epoch = 0200 train time = 14 s train loss = 0.109447 train acc = 0.0100, test acc = 0.9596


100%|██████████| 201/201 [00:13<00:00, 15.29it/s]


[2023-12-04 23:16:13] Evaluate_01: epoch = 0200 train time = 13 s train loss = 0.118280 train acc = 0.0100, test acc = 0.9543


...

[2023-12-04 23:43:08] Evaluate_17: epoch = 0200 train time = 12 s train loss = 0.089454 train acc = 0.0100, test acc = 0.9623


100%|██████████| 201/201 [00:12<00:00, 16.39it/s]


[2023-12-04 23:43:20] Evaluate_18: epoch = 0200 train time = 12 s train loss = 0.093776 train acc = 0.0100, test acc = 0.9655


100%|██████████| 201/201 [00:12<00:00, 15.51it/s]


[2023-12-04 23:43:33] Evaluate_19: epoch = 0200 train time = 12 s train loss = 0.082257 train acc = 0.0100, test acc = 0.9553
Evaluate 20 random ConvNet, mean = 0.9616 std = 0.0030
-------------------------
[2023-12-04 23:43:34] iter = 0100, loss = 0.9031


In [ ]:
print(real_r_test_acc)
print(real_r_train_acc)

[('ConvNet', 0.95686), ('ConvNet', 0.9619699999999998), ('ConvNet', 0.9599500000000001), ('ConvNet', 0.9628650000000001), ('ConvNet', 0.96296), ('ConvNet', 0.9616199999999999)]
[('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01), ('ConvNet', 0.01)]


### Train Different model with Syntheic Dataset

#### ConvNet

In [ ]:
Net = "ConvNet"
model = utils.get_network(Net, channel, num_classes, im_size).to(device)
print(model)
args = datm_arguments(device, 10)
args.lr_net = 0.01
args.epoch_eval_train = 100
args.parall_eva = False
args.dc_aug_param = utils.get_daparam(args.dataset, model, NET, ipc = args.ipc)
args.dsa_strategy = args.dc_aug_param['strategy']

def SoftCrossEntropy(inputs, target, reduction='average'):
    input_log_likelihood = -F.log_softmax(inputs, dim=1)
    target_log_likelihood = F.softmax(target, dim=1)
    batch = inputs.shape[0]
    loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
    return loss

criterion = SoftCrossEntropy

image_syn, label_syn = copy.deepcopy(spc_best_data[0][0]), copy.deepcopy(spc_best_data[0][1]).float()
_, acc_train, acc_test = evaluate_synset(1,model ,image_syn, label_syn, testloader, args, train_criterion=criterion)
print(acc_train)
print(acc_test)

ConvNet(
  (features): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3))
    (1): GroupNorm(128, 128, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(128, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(128, 128, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Linear(in_features=2048, out_features=10, bias=True)
)


100%|██████████| 101/101 [00:04<00:00, 23.60it/s]

[2023-12-05 02:58:56] Evaluate_01: epoch = 0100 train time = 4 s train loss = 0.472251 train acc = 0.0100, test acc = 0.9380
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
0.938


In [ ]:
Net = "ConvNet"
model = utils.get_network(Net, channel, num_classes, im_size).to(device)
print(model)
args = datm_arguments(device, 10)
args.lr_net = 0.01
args.epoch_eval_train = 100
args.parall_eva = False
args.dc_aug_param = utils.get_daparam(args.dataset, model, NET, ipc = args.ipc)
args.dsa_strategy = args.dc_aug_param['strategy']

def SoftCrossEntropy(inputs, target, reduction='average'):
    input_log_likelihood = -F.log_softmax(inputs, dim=1)
    target_log_likelihood = F.softmax(target, dim=1)
    batch = inputs.shape[0]
    loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
    return loss

criterion = SoftCrossEntropy

image_syn, label_syn = copy.deepcopy(real_last_data[0][0]), copy.deepcopy(real_last_data[0][1])
_, acc_train, acc_test = evaluate_synset(1,model ,image_syn, label_syn, testloader, args, train_criterion=criterion)
print(acc_train)
print(acc_test)

ConvNet(
  (features): Sequential(
    (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3))
    (1): GroupNorm(128, 128, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(128, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(128, 128, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (classifier): Linear(in_features=2048, out_features=10, bias=True)
)


100%|██████████| 201/201 [00:07<00:00, 26.26it/s]

[2023-12-05 02:59:18] Evaluate_01: epoch = 0200 train time = 7 s train loss = 0.040060 train acc = 0.0100, test acc = 0.9545
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01

#### VGG11

In [ ]:
Net = "VGG11"
model = utils.get_network(Net, channel, num_classes, im_size).to(device)
print(model)
args = args = datm_arguments(device, 10)
args.lr_net = 0.01
args.epoch_eval_train = 99
args.parall_eva = False
args.dc_aug_param = utils.get_daparam(args.dataset, model, NET, ipc = args.ipc)
args.dsa_strategy = args.dc_aug_param['strategy']

def SoftCrossEntropy(inputs, target, reduction='average'):
    input_log_likelihood = -F.log_softmax(inputs, dim=1)
    target_log_likelihood = F.softmax(target, dim=1)
    batch = inputs.shape[0]
    loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
    return loss

criterion = SoftCrossEntropy

image_syn, label_syn = copy.deepcopy(spc_last_data[0][0]), copy.deepcopy(spc_last_data[0][1])
_, acc_train, acc_test = evaluate_synset(1,model ,image_syn, label_syn, testloader, args, train_criterion=criterion)
print(acc_train)
print(acc_test)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3))
    (1): GroupNorm(64, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(128, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(256, 256, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): GroupNorm(256, 256, eps=1e-05, affine=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16)

100%|██████████| 100/100 [00:06<00:00, 15.75it/s]

[2023-12-05 01:03:17] Evaluate_01: epoch = 0099 train time = 6 s train loss = 0.058570 train acc = 0.0100, test acc = 0.9266
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
0.9266


In [ ]:
Net = "VGG11"
model = utils.get_network(Net, channel, num_classes, im_size).to(device)
print(model)
args = datm_arguments(device, 10)
args.lr_net = 0.01
args.epoch_eval_train = 99
args.parall_eva = False
args.dc_aug_param = utils.get_daparam(args.dataset, model, NET, ipc = args.ipc)
args.dsa_strategy = args.dc_aug_param['strategy']

def SoftCrossEntropy(inputs, target, reduction='average'):
    input_log_likelihood = -F.log_softmax(inputs, dim=1)
    target_log_likelihood = F.softmax(target, dim=1)
    batch = inputs.shape[0]
    loss = torch.sum(torch.mul(input_log_likelihood, target_log_likelihood)) / batch
    return loss

criterion = SoftCrossEntropy

image_syn, label_syn = copy.deepcopy(real_best_data[0][0]), copy.deepcopy(real_best_data[0][1])
_, acc_train, acc_test = evaluate_synset(1,model ,image_syn, label_syn, testloader, args, train_criterion=criterion)
print(acc_train)
print(acc_test)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3))
    (1): GroupNorm(64, 64, eps=1e-05, affine=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): GroupNorm(128, 128, eps=1e-05, affine=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): GroupNorm(256, 256, eps=1e-05, affine=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): GroupNorm(256, 256, eps=1e-05, affine=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (16)

100%|██████████| 100/100 [00:06<00:00, 16.04it/s]

[2023-12-05 01:04:43] Evaluate_01: epoch = 0099 train time = 6 s train loss = 0.064985 train acc = 0.0100, test acc = 0.9385
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
0.9385


# Bonus

In [ ]:
ipcs = [20]
sythetic_datas = []
all_r_test_acc = []
last_datas = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
args = datm_arguments(device, 20)
model = utils.get_network(NET,channel,num_classes).to(device)
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(mnist_dataset, mnist_data_path)
NET = "ConvNet"
args.Iteration = 200
for i in ipcs:
  args.ipc = i
  best_data,last_data, r_test_acc,r_train_acc = DATM(args)
  sythetic_datas.append(best_data)
  all_r_test_acc.append(r_test_acc)
  last_datas.append(last_data)


class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images
Expert Dir: ./MNISTbuffers/MNIST/ConvNet
loading file ./MNISTbuffers/MNIST/ConvNet/replay_buffer_2.pt
0.0.0
1.0.0
2.0.0
3.0.0
4.0.0
5.0.0
6.0.0
7.0.0
8.0.0
9.0.0
InitialAcc:1.0
-------------------------
Evaluation
model_train = ConvNet, model_eval = ConvNet, iteration = 0
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.001, 'strategy': 'crop_scale_rotate'}


100%|██████████| 201/201 [00:21<00:00,  9.53it/s]


[2023-12-05 01:06:57] Evaluate_00: epoch = 0200 train time = 21 s train loss = 0.143394 train acc = 0.0050, test acc = 0.9611


100%|██████████| 201/201 [00:21<00:00,  9.36it/s]


[2023-12-05 01:07:18] Evaluate_01: epoch = 0200 train time = 21 s train loss = 0.147654 train acc = 0.0050, test acc = 0.9636


...

[2023-12-05 02:33:11] Evaluate_17: epoch = 0200 train time = 20 s train loss = 0.071706 train acc = 0.0050, test acc = 0.9733


100%|██████████| 201/201 [00:23<00:00,  8.59it/s]


[2023-12-05 02:33:34] Evaluate_18: epoch = 0200 train time = 23 s train loss = 0.059336 train acc = 0.0050, test acc = 0.9692


100%|██████████| 201/201 [00:21<00:00,  9.36it/s]


[2023-12-05 02:33:56] Evaluate_19: epoch = 0200 train time = 21 s train loss = 0.073488 train acc = 0.0050, test acc = 0.9736
Evaluate 20 random ConvNet, mean = 0.9737 std = 0.0014
-------------------------
[2023-12-05 02:33:58] iter = 0200, loss = 0.9217


In [ ]:
print(r_test_acc)

[('ConvNet', 0.9642200000000001), ('ConvNet', 0.9671100000000001), ('ConvNet', 0.9691449999999999), ('ConvNet', 0.969355), ('ConvNet', 0.9703350000000001), ('ConvNet', 0.97054), ('ConvNet', 0.97327), ('ConvNet', 0.9722950000000001), ('ConvNet', 0.9721249999999999), ('ConvNet', 0.9726250000000001), ('ConvNet', 0.9736500000000001)]


In [ ]:
ipcs = [500]
sythetic_datas = []
all_r_test_acc = []
last_datas = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
args = datm_arguments(device, 20)
model = utils.get_network(NET,channel,num_classes).to(device)
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = utils.get_dataset(mnist_dataset, mnist_data_path)
NET = "ConvNet"

for i in ipcs:
  args.ipc = i
  best_data,last_data, r_test_acc,r_train_acc = DATM(args)
  sythetic_datas.append(best_data)
  all_r_test_acc.append(r_test_acc)
  last_datas.append(last_data)

Train ZCA


100%|██████████| 60000/60000 [00:07<00:00, 7816.21it/s]


Test ZCA


100%|██████████| 10000/10000 [00:01<00:00, 9563.70it/s]


<class 'kornia.enhance.zca.ZCAWhitening'>
class c = 0: 5923 real images
class c = 1: 6742 real images
class c = 2: 5958 real images
class c = 3: 6131 real images
class c = 4: 5842 real images
class c = 5: 5421 real images
class c = 6: 5918 real images
class c = 7: 6265 real images
class c = 8: 5851 real images
class c = 9: 5949 real images
Expert Dir: ./MNISTbuffers/MNIST/ConvNet
loading file ./MNISTbuffers/MNIST/ConvNet/replay_buffer_0.pt
0.0.0
1.0.0
2.0.0
3.0.0
4.0.0
5.0.0
6.0.0
7.0.0
8.0.0
9.0.0
InitialAcc:1.0
-------------------------
Evaluation
model_train = ConvNet, model_eval = ConvNet, iteration = 0
DC augmentation parameters: 
 {'crop': 4, 'scale': 0.2, 'rotate': 45, 'noise': 0.001, 'strategy': 'crop_scale_rotate'}


...

100%|██████████| 201/201 [00:25<00:00,  8.03it/s]


[2023-12-01 00:03:53] Evaluate_03: epoch = 0200 train time = 25 s train loss = 0.671430 train acc = 0.0040, test acc = 0.9361


100%|██████████| 201/201 [00:23<00:00,  8.57it/s]


[2023-12-01 00:04:16] Evaluate_04: epoch = 0200 train time = 23 s train loss = 0.650659 train acc = 0.0040, test acc = 0.9385
Evaluate 5 random ConvNet, mean = 0.9368 std = 0.0010
-------------------------


OutOfMemoryError: ignored

In [ ]:
print(all_r_test_acc)
print(max(all_r_test_acc))